## FNED and FPED Implementation

In [2]:
import pandas as pd
import numpy as np
import math

In [196]:
data_dir = "imdb_mutant"
dfm = pd.read_csv("../data/" + data_dir + "/male/test.csv", header=None, sep="\t", names=["label", "text", "template"])
dff = pd.read_csv("../data/" + data_dir + "/female/test.csv", header=None, sep="\t", names=["label", "text", "template"])

In [197]:
output_dir = "exp1_on_imdb"

result_dir = "../result/" + output_dir + "/"

rfm = result_dir + "results_data_male.txt"
rff = result_dir + "results_data_female.txt"

mpred = []
file = open(rfm)
lines = file.readlines()
for l in lines :
    mpred.append(int(l))
file.close()

print(len(mpred))

fpred = []
file = open(rff)
lines = file.readlines()
for l in lines :
    fpred.append(int(l))
file.close()

print(len(fpred))

139300
139300


In [198]:
dfm["pred"] = mpred
dff["pred"] = fpred

In [199]:
dfm.head()

,label,text,template,pred
0,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",1
1,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",1
2,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",1
3,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",1
4,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",1


In [200]:
df = pd.concat([dfm, dff])

In [201]:
df

,label,text,template,pred
0,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",1
1,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",1
2,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",1
3,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",1
4,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",1
...,...,...,...,...
139295,1,"First, I'm a huge Melanie fan. I grew up knowi...","First, I'm a huge Buddy Holly fan. I grew up k...",1
139296,1,"First, I'm a huge Tanisha fan. I grew up knowi...","First, I'm a huge Buddy Holly fan. I grew up k...",1
139297,1,"First, I'm a huge Nancy fan. I grew up knowing...","First, I'm a huge Buddy Holly fan. I grew up k...",1
139298,1,"First, I'm a huge Tia fan. I grew up knowing w...","First, I'm a huge Buddy Holly fan. I grew up k...",1


In [202]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

y_test = df["label"]
y_pred = df["pred"]
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print(cm)

              precision    recall  f1-score   support

           0       0.86      0.95      0.90    139640
           1       0.94      0.85      0.89    138960

    accuracy                           0.90    278600
   macro avg       0.90      0.90      0.90    278600
weighted avg       0.90      0.90      0.90    278600

[[132186   7454]
 [ 20923 118037]]


In [203]:
y_test = dfm["label"]
y_pred = dfm["pred"]
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print(cm)

              precision    recall  f1-score   support

           0       0.86      0.95      0.90     69820
           1       0.94      0.85      0.89     69480

    accuracy                           0.90    139300
   macro avg       0.90      0.90      0.90    139300
weighted avg       0.90      0.90      0.90    139300

[[66044  3776]
 [10348 59132]]


In [204]:
y_test = dff["label"]
y_pred = dff["pred"]
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print(cm)

              precision    recall  f1-score   support

           0       0.86      0.95      0.90     69820
           1       0.94      0.85      0.89     69480

    accuracy                           0.90    139300
   macro avg       0.90      0.90      0.90    139300
weighted avg       0.90      0.90      0.90    139300

[[66142  3678]
 [10575 58905]]


In [205]:
def calculate_fpr(cm) :
    negative = np.sum(cm[0])
    fp = cm[0][1]
    fpr = fp / negative
    return fpr

def calculate_fnr(cm) :
    negative = np.sum(cm[0])
    fn = cm[1][0]
    fnr = fn / negative
    return fnr

print(calculate_fpr(cm))
print(calculate_fnr(cm))

0.05267831566886279
0.15146089945574334


In [206]:
y_test = df["label"]
y_pred = df["pred"]
cm = confusion_matrix(y_test, y_pred)
global_fpr = calculate_fpr(cm)
global_fnr = calculate_fnr(cm)
print("global fpr: ", global_fpr)
print("global fnr: ", global_fnr)

d = [dfm, dff]
fped = 0
fned = 0
for _d in d :
    y_test = _d["label"]
    y_pred = _d["pred"]
    cm = confusion_matrix(y_test, y_pred)
    fpr = calculate_fpr(cm)
    fnr = calculate_fnr(cm)
    fped += abs(global_fpr - fpr)
    fned += abs(global_fnr - fnr)
    print("fpr: ", fpr)
    print("fnr: ", fnr)


print(fped)
print(fned)

global fpr:  0.053380120309366945
global fnr:  0.14983529074763677
fpr:  0.054081924949871095
fnr:  0.14820968203953022
fpr:  0.05267831566886279
fnr:  0.15146089945574334
0.001403609281008307
0.00325121741621312
